loading basic packages

In [406]:
# always yielding a real result, even dividing two integers
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import datetime as dt
from dateutil.relativedelta import relativedelta
from functools import reduce
import operator
import datetime
from scipy.optimize import minimize
import math

# plot inline
%matplotlib inline

loading working data

In [237]:
data_arq = 'data.xlsx'
holidays_cdi = pd.read_excel(data_arq,'holidays_cdi')
copom_meetings = pd.read_excel(data_arq,'copom_meetings')
di1_series = pd.read_excel(data_arq,'di1_series')

In [79]:
di1s=pd.DataFrame(di1_series.values,columns=['MarketDate','MaturityDate','MaturityId','Open','Close','High','Low','Avg','Settlement','SettlementD1','TotalContracts','BusinessDaysToMaturity','MaturityNumber'],index=di1_series['MarketDate'].values)

In [251]:
refdate = datetime.datetime(2015, 4, 2)

In [252]:
temp = di1s.loc[refdate,["MaturityDate","Close"]]

In [253]:
df_ffc = pd.DataFrame(temp['Close'].values,index=temp['MaturityDate'],columns=['rm'])

In [254]:
df_ffc = df_ffc.loc[df_ffc['rm'] > 0]

In [255]:
df_copom = pd.DataFrame(copom_meetings['InterestTarget'].values,index=copom_meetings['MeetingDate'].values,columns=['InterestTarget'])

In [256]:
temp = df_copom['InterestTarget'][refdate:]

In [257]:
df_copom = pd.DataFrame(temp.values,index=temp.index,columns=['InterestTarget'])

In [258]:
df_ffc = pd.concat([df_ffc,df_copom],axis=1)

In [271]:
df_ffc.loc[pd.notnull(df_ffc['rm']),'Tipo']='DI'

In [274]:
df_ffc.loc[pd.isnull(df_ffc['Tipo']),'Tipo']='COPOM'

In [390]:
model=pd.read_excel(data_arq,'model')

In [422]:
def model_getx0(model):
    x0=np.zeros(len(model.index), dtype=float)
    rm=model.loc[0,'rm']
    for i in range(len(model.index)):
        if not math.isnan(model.loc[i,'rm']):
            rm=model.loc[i,'rm']
            
        x0[i]=rm
    return x0

In [425]:
x0=model_getx0(model)

In [428]:
def model_calculate_error(model,x):
    for i in range(len(model.index)):
        if(model.loc[i]['same fwd?']==False):
            model.loc[i,'fwd']=x[i]
        else:
            model.loc[i,'fwd']=model.loc[i-1]['fwd']
        
        if(i==0):
            model.loc[i,'r']=model.loc[i]['rm']
        else:
            r=(1+model.loc[i-1]['r']/100)**(model.loc[i-1]['t']/252)
            r*=((1+model.loc[i]['fwd']/100)**((model.loc[i]['t']-model.loc[i-1]['t'])/252))
            model.loc[i,'r']=(r**(252/model.loc[i]['t'])-1)*100
        
    model['|rm-r|']=abs(model['rm']-model['r'])
    return model['|rm-r|'].sum()

In [429]:
model_calculate_error(model,x0)

7.434823061301769

In [433]:
model_to_minimize = lambda x: model_calculate_error(model,x)
res=minimize(model_to_minimize,x0,method='nelder-mead',options={'xtol': 1e-8, 'disp': True})

KeyboardInterrupt: 